# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [20]:
# importing needed libraries for the entire notebook in one go
# %matplotlib inline
# %matplotlib widget
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from itertools import combinations
import eof
import geemap
import csv
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, box


# only need these two if you plan on searching andrequesting for data from within this notebook
# in my opinion, while it may take longer, it is much easier to use the ASF Vertex Website
# that said, if you are monitoring a reoccuring site and just need more imagery, searching using the notebook would be best
import asf_search as asf
import hyp3_sdk as sdk

In [21]:
def project_dir(proj_name):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    """

    #creates file on your desktop containing the work of this notebook
    work_dir = os.path.join(os.path.expanduser('~'),f'Desktop/{proj_name}')
    os.makedirs(work_dir, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'interferometry')
    os.makedirs(if_dir, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'time_series')
    os.makedirs(ts_dir, exist_ok=True)
    

    xmls_dir = os.path.join(if_dir,'xmls')
    os.makedirs(xmls_dir, exist_ok=True)
    
    xmldirectories=[]
    for dir in ['topsApp', 'reference', 'secondary']:
        dirpath = os.path.join(xmls_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        xmldirectories.append(dirpath)

    ifdirectories=[]
    for dir in ['reference', 'secondary', 'orbits', 'work']:
        dirpath = os.path.join(if_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        ifdirectories.append(dirpath)

    tsdirectories=[]
    for dir in ['baseline', 'reference', 'merged', 'secondaries', 'mintpy']:
        dirpath = os.path.join(ts_dir, dir)
        os.makedirs(dirpath, exist_ok=True)
        if dir == 'merged':
                geomref_dir = os.path.join(dirpath,'geom_reference')
                os.makedirs(geomref_dir, exist_ok=True)

                interfer_dir = os.path.join(dirpath,'interferograms')
                os.makedirs(interfer_dir, exist_ok=True)

                tsdirectories.append(geomref_dir)
                tsdirectories.append(interfer_dir)

        tsdirectories.append(dirpath)

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [22]:
def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet

In [23]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
# function to write the 
def ref_sec_xml(slc_zips_list, slc_zips_dirs):
    for i in range(len(slc_zips_dirs)):
        for j, type in enumerate(['reference', 'secondary']):
            imset = ET.Element('component', name=type)
            safe = ET.SubElement(imset, 'property', name='safe').text = slc_zips_dirs[i]
            out_dir =ET.SubElement(imset, 'property', name='output directory').text = ifdirectories[j]
            orbit_dir =ET.SubElement(imset, 'property', name='orbit directory').text = ifdirectories[2]
            roi = ET.SubElement(imset, 'property', name='region of interest').text = str(isce_aoi)
            tree = ET.ElementTree(imset)
            tree.write(os.path.join(xmldirectories[j+1], f'{slc_zips_list[i][17:25]}{type[:3]}.xml'))

In [24]:
# function to create the topsApp.xml files for topsInSAR
def topsApp_xml(triplet_dict):
    for key in triplet_dict:
        for pair in triplet_dict[key]:
            data = ET.Element('topsApp')
            tinsar = ET.SubElement(data, 'component', name='topsinsar')
            snsr_name = ET.SubElement(tinsar, 'property', name='Sensor name').text = 'SENTINEL1'
            ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
            xml_file = ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{pair[0][17:25]}ref.xml")
            sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
            xml_file = ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{pair[1][17:25]}sec.xml")
            
            swathnums = ET.SubElement(tinsar, 'property', name='swaths').text=str([1,2])

            azi_looks = ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(6)
            range_looks = ET.SubElement(tinsar, 'property', name='range looks').text = str(2)
            
            phasefilt = ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)
            
            
            unwrap_yn = ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True) # or False
            unwrap_name = ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf' # grass, icu, snaphu, snaphu_mcf, downsample_snap
            
            do_iono_corr = ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = str(False)
            apply_iono_corr = ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = str(False)
            
            #choose from: ['subband', 'rawion', 'grd2ion', 'filt_gaussian', 'ionosphere_shift', 'ion2grd', 'esd']
            start_iono_corr = ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'filt_gaussian'
            end_iono_corr = ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'
    
            geocode = ET.SubElement(tinsar, 'property', name='geocode bounding box').text = str(isce_aoi)
            
            tree = ET.ElementTree(data)
            tree.write(os.path.join(xmldirectories[0], f"{pair[0][17:25]}_{pair[1][17:25]}topsApp.xml"))

# Establish working directroy and data paths

In [25]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'Harris_etal_2023'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name)

In [26]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

triplet_dict, test_triplet = get_triplets(slc_zips_list)

In [27]:
# examine each triplet if you would like
# this shows the tuple that stores the string filenmem of the SLC .zip files
# in each triplet there will be 4 SLC pairs = 4 interferograms
# the first makes if_ij = the multilooked wrapped interferometric phase between SLC i and j
# the second makes if_jk = the multilooked wrapped interferometric phase between SLC j and k
# the third makes if_ki = the multilooked wrapped interferometric phase between SLC k and i
# where i, j, k are a time-series of SLC images
# choose a triplet to examine its pairs
# closure phase = if_ij+if_jk+if_ki

# triplet_choice = 'triple_1'
triplet_choice = f'triplet_{len(triplet_dict)}'  # uncomment to just check the last triplet in the dict

if_ij = triplet_dict[triplet_choice][0]
if_jk = triplet_dict[triplet_choice][1]
if_ki = triplet_dict[triplet_choice][2]

print()
print(f'Pair 1:{if_ij}')
print(f'Pair 2:{if_jk}')
print(f'Pair 3:{if_ki}')


Pair 1:('S1A_IW_SLC__1SDV_20180208T001821_20180208T001848_020508_023144_A39F.zip', 'S1A_IW_SLC__1SDV_20180220T001821_20180220T001848_020683_0236D8_9211.zip')
Pair 2:('S1A_IW_SLC__1SDV_20180220T001821_20180220T001848_020683_0236D8_9211.zip', 'S1A_IW_SLC__1SDV_20180328T001822_20180328T001849_021208_024782_46FD.zip')
Pair 3:('S1A_IW_SLC__1SDV_20180328T001822_20180328T001849_021208_024782_46FD.zip', 'S1A_IW_SLC__1SDV_20180208T001821_20180208T001848_020508_023144_A39F.zip')


# Get bbox for your area of interest in the SLC images, faster processing with less data

In [28]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [29]:
# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(m.draw_features[0].getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
isce_aoi_polygon = Polygon(vertices)

#create isce compatibile bbox
isce_aoi = [min([easting for northing, easting in vertices[:]]), max([easting for northing, easting in vertices[:]]), min([northing for northing, easting in vertices[:]]), max([northing for northing, easting in vertices[:]])]
print(isce_aoi)

[29.813242, 30.089296, -93.544464, -93.309631]


# Get s-1 orbit files (.EOF) using sentineleof library

In [25]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create Sentinel-1 SLC .xml files
- One reference and one secondary for each image

In [30]:
ref_sec_xml(slc_zips_list, slc_zips_dirs)

# Create topsApp.xml file
- one for each interferogram in each triplet
- (N_triplets * 4) - (N_triplets - 1)   ??

In [31]:
topsApp_xml(triplet_dict) # stores these in xmldirectories[0]

In [32]:
# check to make sure all topsApp.xml files were created
# if the cell prints somehting, not all topsApp.xmls were created

topsAppxml_triplet_dict = {}
for key, triplets in triplet_dict.items():
    topsapppairs = (f'{triplets[0][0][17:25]}_{triplets[0][1][17:25]}topsApp.xml', 
             f'{triplets[1][0][17:25]}_{triplets[1][1][17:25]}topsApp.xml', 
             f'{triplets[2][0][17:25]}_{triplets[2][1][17:25]}topsApp.xml'
             )
    topsAppxml_triplet_dict[key] = topsapppairs


for i in range(1, len(triplet_dict)+1):
    topsapptrip = topsAppxml_triplet_dict[f'triplet_{i}']
    slctrip = triplet_dict[f'triplet_{i}']
    for j, slcpair in enumerate(slctrip):
        slcdate = f'{slcpair[0][17:25]}_{slcpair[1][17:25]}'
        topsappdate = topsapptrip[j][:-11]
        if slcdate != topsappdate:
            print(slcdate, topsappdate)

# topsApp interferometry (finally!)

In [33]:
# should import isce toolbox and print the version of it here
# if nothing prints or nothing imports, you need to reinstall isce2

import isce
isce.version.release_version

'2.6.3'

Below is the cell that performs the InSAR processing. Still need to work on a couple things:

1. ISCE2 CLI allows for running separate steps and saving pickles between steps, want to figure out how to do that in the IDE
2. Iterating through each .xml to create the interferograms
3. Saving XMLs to appropriate storage locations for MintPy time-series
4. runnings prep_isce.py from mintPy to prep the topsApp results

Need to look into the difference between topsStack and topsApp results, as Mintpy docs only specified topsStack. Presumably, topsStack stacks the interferograms in an easier to use format for mintpy, compared to the individual interferograms generated by topsApp

In [34]:
# this is used to change directory into the storage directory for all interferograms
# should be used, else will save results to github repo
os.chdir(ifdirectories[3])
print(ifdirectories[3])

/home/wcc/Desktop/Harris_etal_2023/interferometry/work


In [55]:
from topsApp import TopsInSAR
insar = TopsInSAR(name="topsApp", cmdline=os.path.join(xmldirectories[0], os.listdir(xmldirectories[0])[0]))
insar.configure()

2024-09-24 10:28:42,622 - isce.insar - INFO - ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 
ISCE VERSION = 2.6.3, RELEASE_SVN_REVISION = ,RELEASE_DATE = 20230418, CURRENT_SVN_REVISION = 


Steps include:
- runPreprocessor
- runComputeBaseline
- verifyDEM
- verifyGeocodeDEM
- runTopo
- runSubsetOverlaps
- runCoarseOffsets
- runCoarseResamp
- runOverlapIfg
- runPrepESD
- runESD
- runRangeCoreg
- runFineOffsets
- runFineResamp
- runIon
- runBurstIfg
- runMergeBursts
- runFilter
- runGeocode
- runDenseOffsets
- runOffsetFilter

I think mine is breaking at runfilter, reinstalled isce2 to check

In [56]:
insar.startup()

None
The currently supported sensors are:  ['SENTINEL1']


In [57]:
insar.runPreprocessor()

Input XML files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/annotation/s1a-iw1-slc-vv-20170601t001818-20170601t001843-016833-01bfc6-004.xml']
Input TIFF files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/measurement/s1a-iw1-slc-vv-20170601t001818-20170601t001843-016833-01bfc6-004.tiff']
Manifest files:  ['/vsizip//home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.zip/S1A_IW_SLC__1SDV_20170601T001818_20170601T001845_016833_01BFC6_9708.SAFE/manifest.safe']
MANS:  /home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2017_2018/S1A_IW_SLC__1SDV_20170601T001818

In [60]:
insar.runComputeBaseline()

Estimated burst offset:  0
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Estimated burst offset:  0
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
2024-09-24 10:31:48,471 - isce.topsinsar.runPreprocessor - INFO - 
####################################################################################################
    runComputeBaseline
----------------------------------------------------------------------------------------------------
baseline.IW-1 Bpar at midrange for first common burst = -39.6352604864038
baseline.IW-1 Bpar at midrange for last common burst = -37.32465596659518
baseline.IW-1 Bperp at midrange for first common burst = 76.0456386716753
baseline.IW-1 Bperp at midrange for last common burst = 72.21274432563119
baseline.IW-1 First common burst in reference = 0
baseline.IW-1 First common burst in secondary = 0
baseline.IW-1 Last common burst in reference = 9
baseline.IW-1 Last common burst in secondary = 9
baseline.IW-1 Number of burs

In [62]:
#Verify whether user defined  a dem component.  If not, then download
# SRTM DEM.

#probably where I enter custom DEM, for now not worrying about it

insar.verifyDEM()

'demLat_N29_N31_Lon_W096_W093.dem.wgs84'

In [63]:
insar.verifyGeocodeDEM()

'demLat_N29_N31_Lon_W096_W093.dem.wgs84'

In [64]:
insar.runTopo()

2024-09-24 10:37:59,911 - isce.zerodop.topozero - WARNING - Default Peg heading set to: -0.22132091182996402
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 1 
798980	2.32956	
GDAL open (R): demLat_N29_N31_Lon_W096_W093.dem.wgs84.vrt
C pointer already created. Finalize and recreate if image dimensions changed.
C pointer already created. Finalize and recreate if image dimensions changed.
API open (WR): geom_reference/IW1/lat_01.rdr
API open (WR): geom_reference/IW1/lon_01.rdr
API open (WR): geom_reference/IW1/hgt_01.rdr
API open (WR): geom_reference/IW1/los_01.rdr
 Initializing biquintic interpolator
 Orbit interpolation method: hermite
 Max threads used:           24
 DEM parameters:
 Dimensions:        10800        7200
 Top Left:   -96.000000000000000        31.000000000000000     
 Spacing:    2.7777777777777778E-004  -2.7777777777777778E-004
 Lon:   -96.000000000000000       -93.000277777777782     
 Lat:    29.000277777777779        31.000000000000000     
  
 Estimate

In [65]:
insar.runSubsetOverlaps()

Top:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_01_02.slc']
Bottom:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_bot_01_02.slc']
Top:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_01_02.slc', '/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_02_03.slc']
Bottom:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_bot_01_02.slc', '/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_bot_02_03.slc']
Top:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_01_02.slc', '/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_02_03.slc', '/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_03_04.slc']
Bottom:  ['/home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_bot_0

In [66]:
insar.runCoarseOffsets()

2024-09-24 10:43:53,171 - isce.topsinsar.coarseoffsets - INFO - Overlap pair 0, IW-1: Burst 0 of reference matched with Burst 0 of secondary
2024-09-24 10:43:53,180 - isce.topsinsar.coarseoffsets - INFO - Additional time offset applied in geo2rdr: 0.0 secs
2024-09-24 10:43:53,180 - isce.topsinsar.coarseoffsets - INFO - Additional range offset applied in geo2rdr: 0.0 m
Turning off bistatic delay correction term by default.
API open (WR): coarse_offsets/overlaps/IW1/range_top_01_02.off
API open (WR): coarse_offsets/overlaps/IW1/azimuth_top_01_02.off
GDAL open (R): geom_reference/overlaps/IW1/hgt_01_02.rdr.vrt
GDAL open (R): geom_reference/overlaps/IW1/lat_01_02.rdr.vrt
GDAL open (R): geom_reference/overlaps/IW1/lon_01_02.rdr.vrt
 Orbit interpolation method: hermite
 threads          24
 Starting Acquisition time:    1097.5007220000000     
 Stop Acquisition time:    1100.5655564433000     
 Azimuth line spacing in secs:    2.0555562999999980E-003
 Near Range in m:    798980.13693783211  

In [68]:
insar.runCoarseResamp()

Shifts for swath IW-1: {0: 0, 1: -1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Chi squared: 0.000173
Polynomial Order: 5 - by - 3 
13192.4	-762.006	72.497	1.48892	
-51284.1	2716.22	-192.598	-4.94847	
49840.5	-2400.66	101.459	3.44755	
-9.63225e-10	-2.85352e-10	5.5769e-11	0	
1.27099e-09	8.36348e-11	0	0	
-5.54588e-10	0	0	0	
Misfit radians - Max: 0.0006334351801342564 , Min : -0.00034247628900629934 
2024-09-24 10:44:16,349 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1492 which is the number of lines in the slc image.
2024-09-24 10:44:16,350 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the default value 21489 which is the width of the slc image.
2024-09-24 10:44:16,350 - isce.stdproc.resamp_slc - WARNING - No flattening requested
No Range Carrier provided.
Assuming zero range carrier.
GDAL open (R): /home/wcc/Desktop/Harris_etal_2023/inter

In [69]:
insar.runOverlapIfg()

GDAL open (R): coarse_interferogram/overlaps/IW1/burst_top_01_02.int.vrt
API open (WR): coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
GDAL close: coarse_interferogram/overlaps/IW1/burst_top_01_02.int.vrt
API close:  coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
API open (R): coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
API close:  coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
Writing geotrans to VRT for coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
Output:  coarse_interferogram/overlaps/IW1/burst_top_01_02.6alks_2rlks.int
GDAL open (R): coarse_interferogram/overlaps/IW1/burst_bot_01_02.int.vrt
API open (WR): coarse_interferogram/overlaps/IW1/burst_bot_01_02.6alks_2rlks.int
GDAL close: coarse_interferogram/overlaps/IW1/burst_bot_01_02.int.vrt
API close:  coarse_interferogram/overlaps/IW1/burst_bot_01_02.6alks_2rlks.int
API open (R): coarse_interferogram/overlaps/IW1/burst_bot_01_02.6alks

In [70]:
##Prepare for ESD
insar.runPrepESD()

Relative shifts for swath 1:
{0: 0, 1: -1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}
GDAL open (R): ESD/overlap_IW1_01.int.vrt
API open (WR): ESD/overlap_IW1_01.5alks_15rlks.int
GDAL close: ESD/overlap_IW1_01.int.vrt
API close:  ESD/overlap_IW1_01.5alks_15rlks.int
API open (R): ESD/overlap_IW1_01.5alks_15rlks.int
API close:  ESD/overlap_IW1_01.5alks_15rlks.int
Writing geotrans to VRT for ESD/overlap_IW1_01.5alks_15rlks.int
Output:  ESD/overlap_IW1_01.5alks_15rlks.int
2024-09-24 10:53:31,192 - root - DEBUG - Creating readonly ISCE mmap with 
file = ESD/overlap_IW1_01.5alks_15rlks.int 
bands = 1 
width = 1426 
length = 21 
scheme = BIP 
dtype = CFLOAT 

GDAL open (R): ESD/freq_IW1_01.bin.vrt
API open (WR): ESD/freq_IW1_01.5alks_15rlks.bin
GDAL close: ESD/freq_IW1_01.bin.vrt
API close:  ESD/freq_IW1_01.5alks_15rlks.bin
API open (R): ESD/freq_IW1_01.5alks_15rlks.bin
API close:  ESD/freq_IW1_01.5alks_15rlks.bin
Writing geotrans to VRT for ESD/freq_IW1_01.5alks_15rlks.bin
Output:  ESD/freq_I

In [71]:
# run ESD
insar.runESD()

Previous version of ESD/combined_IW1.int found. Cleaning ...
Previous version of ESD/combined_freq_IW1.bin found. Cleaning ...
Previous version of ESD/combined_IW1.cor found. Cleaning ...
Previous version of ESD/combined_IW1.off found. Cleaning ...
Previous version of ESD/combined_IW2.int found. Cleaning ...
Previous version of ESD/combined_freq_IW2.bin found. Cleaning ...
Previous version of ESD/combined_IW2.cor found. Cleaning ...
Previous version of ESD/combined_IW2.off found. Cleaning ...
2024-09-24 10:53:41,434 - isce.topsinsar.esd - INFO - 
####################################################################################################
    runESD
----------------------------------------------------------------------------------------------------
esd.Mean = 4.6780193598933986e-05
esd.Median = 2.8551136892929208e-05
esd.Std = 0.002605069599269444
esd.coherence threshold = 0.85
esd.number of coherent points = 2072
#################################################################

In [72]:
###Estimate range misregistration
insar.runRangeCoreg()

GDAL open (R): /home/wcc/Desktop/Harris_etal_2023/interferometry/reference/overlaps/IW1/burst_top_01_02.slc.vrt
GDAL open (R): coarse_coreg/overlaps/IW1/burst_top_01_02.slc.vrt
Scale Factor in Range:  1.0
Scale Factor in Azimuth:  1.0
Skip Sample Down: 3
Skip Sample Across: 492
 Input Bands:            1           1
  XXX start timer
 wisdomFile, length =            0
CAUTION - Requesting very large search window pull in
Reference Window Size is                      64 sample pixels
Number of Search Pixels is                    16 sample pixels
The rule of thumb is that the search window pull in is at least 5
pixels and is less than the reference window size divided by 5. 
Suggested Number of Search Pixels is          11 sample pixels
 
 
CAUTION - Requesting very large search window pull in
Reference Window Size is                      32 line pixels
Number of Search Pixels is                    16 line pixels
The rule of thumb is that the search window pull in is at least 5
pixels an

In [73]:
###Estimate fine offsets
insar.runFineOffsets()

2024-09-24 10:57:40,565 - isce.topsinsar.fineoffsets - INFO - IW1 - Burst 0 of reference matched with Burst 0 of secondary
GDAL open (R): geom_reference/IW1/lat_01.rdr.vrt
GDAL open (R): geom_reference/IW1/lon_01.rdr.vrt
2024-09-24 10:57:40,573 - isce.topsinsar.fineoffsets - INFO - Additional time offset applied in geo2rdr: 5.8688469312423004e-08 secs
GDAL open (R): geom_reference/IW1/hgt_01.rdr.vrt
2024-09-24 10:57:40,573 - isce.topsinsar.fineoffsets - INFO - Additional range offset applied in geo2rdr: 0.013080644217043538 m
Turning off bistatic delay correction term by default.
API open (WR): fine_offsets/IW1/range_01.off
API open (WR): fine_offsets/IW1/azimuth_01.off
 Orbit interpolation method: hermite
 threads          24
 Starting Acquisition time:    1097.5007220000000     
 Stop Acquisition time:    1100.5655564433000     
 Azimuth line spacing in secs:    2.0555562999999980E-003
 Near Range in m:    798980.12385718792     
 Far  Range in m:    849037.75457819074     
 Range sa

In [74]:
###Resample secondary bursts
insar.runFineResamp()

Shifts IW-1:  {0: 0, 1: -1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Chi squared: 0.000173
Polynomial Order: 5 - by - 3 
13192.4	-762.006	72.497	1.48892	
-51284.1	2716.22	-192.598	-4.94847	
49840.5	-2400.66	101.459	3.44755	
-9.63225e-10	-2.85352e-10	5.5769e-11	0	
1.27099e-09	8.36348e-11	0	0	
-5.54588e-10	0	0	0	
Misfit radians - Max: 0.0006334351801342564 , Min : -0.00034247628900629934 
2024-09-24 11:00:00,166 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1492 which is the number of lines in the slc image.
2024-09-24 11:00:00,166 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the default value 21489 which is the width of the slc image.
2024-09-24 11:00:00,166 - isce.stdproc.resamp_slc - WARNING - No flattening requested
No Range Carrier provided.
Assuming zero range carrier.
GDAL open (R): /home/wcc/Desktop/Harris_etal_2023/interferometry

In [ ]:
###calculate ionospheric phase
insar.runIon()

In [ ]:
###Create burst interferograms
insar.runBurstIfg()

In [ ]:
####Merge bursts into single files

#could see if I even need to merge them at all?

insar.runMergeBursts()

In [ ]:
###Filter the interferogram
insar.runFilter()

In [ ]:
#add water mask to coherence and interferogram
#insar.runMaskImages()

In [ ]:
# Unwrap ?
insar.runUnwrapper()

In [ ]:
# Geocode
insar.runGeocode(insar.geocode_list, insar.do_unwrap, insar.geocode_bbox)

In [ ]:
#Dense offsets
insar.runDenseOffsets()

In [ ]:
#Filter offsets
insar.runOffsetFilter()

In [ ]:
#Geocode offsets
insar.runGeocode(insar.off_geocode_list, False, insar.geocode_bbox, True)

In [54]:
insar.endup()

2024-09-24 10:28:22,366 - isce.insar - INFO - Total Time: 1 seconds


In [ ]:
# maybe put something here that will move the files generated
# put them into the example directory that Mintpy needs for topsStack



# Working to make it so that the above saves into needed file directories for Mintpy

https://mintpy.readthedocs.io/en/stable/dir_structure/

Essentially just needs another function for:
1. runnings prep_isce.py on the data produced above
2. move all needed data created from ISCE2 into the tsdirectories according to the above link

(this may need to be done in the mintpy environment, so I may end up including this in the time series ipynb, or a short .py to run on it's own)